In [94]:
import warnings
warnings.filterwarnings(action='ignore')

from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()


In [95]:
type(iris)

sklearn.utils._bunch.Bunch

In [96]:
iris.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [97]:
dir(iris)

['DESCR',
 'data',
 'data_module',
 'feature_names',
 'filename',
 'frame',
 'target',
 'target_names']

In [98]:
irisDF = pd.DataFrame(data=iris.data, columns=['sl', 'sw', 'pl', 'pw'])
irisDF['target'] = iris.target
irisDF.head()

,sl,sw,pl,pw,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [99]:
from sklearn.model_selection import train_test_split

y = irisDF['target']
X = irisDF.drop(['target'], axis=1)
print(X.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, stratify=y, random_state=1111)
X_train.shape, X_test.shape, y_train.shape, y_test.shape 

(150, 4) (150,)


((120, 4), (30, 4), (120,), (30,))

In [100]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=300, random_state=1111)
kmeans.fit(X_train)
kmeans.labels_

array([1, 0, 1, 2, 2, 1, 0, 1, 1, 0, 1, 2, 0, 0, 1, 1, 0, 2, 0, 0, 1, 2,
       2, 2, 0, 1, 1, 0, 0, 0, 2, 0, 2, 1, 1, 0, 1, 2, 1, 2, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 2, 2, 1, 0, 1, 2, 1, 0, 1, 2, 0, 2, 0, 0, 2, 1, 2,
       0, 0, 2, 0, 0, 0, 0, 2, 1, 2, 1, 1, 2, 0, 0, 0, 1, 0, 2, 1, 0, 0,
       1, 1, 1, 2, 0, 0, 2, 2, 2, 1, 1, 1, 2, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 0, 2, 1, 2])

In [101]:
pred = kmeans.predict(X_train)
pred

array([1, 0, 1, 2, 2, 1, 0, 1, 1, 0, 1, 2, 0, 0, 1, 1, 0, 2, 0, 0, 1, 2,
       2, 2, 0, 1, 1, 0, 0, 0, 2, 0, 2, 1, 1, 0, 1, 2, 1, 2, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 2, 2, 1, 0, 1, 2, 1, 0, 1, 2, 0, 2, 0, 0, 2, 1, 2,
       0, 0, 2, 0, 0, 0, 0, 2, 1, 2, 1, 1, 2, 0, 0, 0, 1, 0, 2, 1, 0, 0,
       1, 1, 1, 2, 0, 0, 2, 2, 2, 1, 1, 1, 2, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 0, 2, 1, 2])

In [105]:
from sklearn.metrics import silhouette_samples, silhouette_score
score_samples = silhouette_samples(X_train, kmeans.labels_)
score_samples.mean()

0.5413260307899953

In [106]:
# irisDF['silhouette_coeff'] = score_samples
average_score = silhouette_score(X_train, pred)
average_score

0.5413260307899953

In [107]:
X_train_copy = X_train.copy()
X_train_copy['cluster'] = pred
X_train_copy.head(3)

,sl,sw,pl,pw,silhouette_coeff,cluster
15,5.7,4.4,1.5,0.4,0.642884,1
79,5.7,2.6,3.5,1.0,0.500428,0
4,5.0,3.6,1.4,0.2,0.844983,1


In [108]:
X_train_copy.groupby('cluster')['silhouette_coeff'].mean()

cluster
0    0.387707
1    0.793667
2    0.472877
Name: silhouette_coeff, dtype: float64

# DBSCAN

In [116]:
from sklearn.cluster import DBSCAN

# irisDF = pd.DataFrame(data=iris.data, columns=['sl', 'sw', 'pl', 'pw'])

dbscan = DBSCAN(eps=0.8, min_samples=8, metric='euclidean')
dbscan_lables = dbscan.fit_predict(iris.data)

irisDF['dbscan_cluster'] = dbscan_lables
irisDF['target'] = iris.target

iris_result = irisDF.groupby('target')['dbscan_cluster'].value_counts()
iris_result

target  dbscan_cluster
0        0                50
1        1                50
2        1                47
        -1                 3
Name: dbscan_cluster, dtype: int64

In [112]:
irisDF

,sl,sw,pl,pw,target,dbscan_cluster
0,5.1,3.5,1.4,0.2,0,0
1,4.9,3.0,1.4,0.2,0,0
2,4.7,3.2,1.3,0.2,0,0
3,4.6,3.1,1.5,0.2,0,0
4,5.0,3.6,1.4,0.2,0,0
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,1
146,6.3,2.5,5.0,1.9,2,1
147,6.5,3.0,5.2,2.0,2,1
148,6.2,3.4,5.4,2.3,2,1
